# Lesson 10: Mixing for FRI

In this lesson, we'll examine how mixing is applied in the FRI (Fast Reed-Solomon IOP) protocol. This is an important step that prepares data for efficient proof of polynomial low degree.

In [ ]:
import numpy as np
from typing import List, Tuple
import matplotlib.pyplot as plt

class FRIMixer:
    """Implements mixing for FRI protocol."""
    
    def __init__(self, domain_size: int):
        self.domain_size = domain_size
        self.omega = np.exp(2j * np.pi / domain_size)  # Root of unity
        
    def generate_mixing_coefficients(self, num_coeffs: int) -> np.ndarray:
        """Generates random coefficients for mixing."""
        return np.random.rand(num_coeffs) + 1j * np.random.rand(num_coeffs)
    
    def fold_polynomial(self, values: np.ndarray, alpha: complex) -> np.ndarray:
        """Performs polynomial folding using alpha.
        
        Args:
            values: Polynomial values
            alpha: Mixing coefficient
            
        Returns:
            np.ndarray: Folded values
        """
        n = len(values)
        half_n = n // 2
        
        # Split into even and odd indices
        even = values[::2]
        odd = values[1::2]
        
        # Perform folding
        domain = np.array([self.omega ** i for i in range(half_n)])
        folded = even + alpha * odd * domain
        
        return folded
    
    def create_fri_layer(self, values: np.ndarray) -> Tuple[np.ndarray, complex]:
        """Creates one FRI layer.
        
        Args:
            values: Current layer values
            
        Returns:
            Tuple[np.ndarray, complex]: New layer and used coefficient
        """
        alpha = self.generate_mixing_coefficients(1)[0]
        next_layer = self.fold_polynomial(values, alpha)
        return next_layer, alpha

## Demonstrating FRI Mixing

In [ ]:
# Create test data
domain_size = 16  # Must be power of two
mixer = FRIMixer(domain_size)

# Create test polynomial (through its values)
x = np.linspace(0, 2*np.pi, domain_size)
initial_values = np.sin(x) + 1j * np.cos(x)

# Create several FRI layers
num_layers = 3
layers = [initial_values]
alphas = []

for _ in range(num_layers):
    next_layer, alpha = mixer.create_fri_layer(layers[-1])
    layers.append(next_layer)
    alphas.append(alpha)

# Visualization
plt.figure(figsize=(15, 10))

# Plot real parts
plt.subplot(2, 1, 1)
for i, layer in enumerate(layers):
    plt.plot(np.real(layer), label=f'Layer {i}')
plt.title('Real Parts of FRI Layers')
plt.xlabel('Index')
plt.ylabel('Value')
plt.grid(True)
plt.legend()

# Plot imaginary parts
plt.subplot(2, 1, 2)
for i, layer in enumerate(layers):
    plt.plot(np.imag(layer), label=f'Layer {i}')
plt.title('Imaginary Parts of FRI Layers')
plt.xlabel('Index')
plt.ylabel('Value')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Compression analysis
print("\nFRI Layer Analysis:")
for i, layer in enumerate(layers):
    print(f"Layer {i}: {len(layer)} values")
    if i > 0:
        print(f"Coefficient α_{i}: {alphas[i-1]:.3f}")

## Analysis of FRI Mixing Properties

1. **Mathematical Properties**:
   - Preservation of polynomial degree during folding
   - Transformation linearity
   - Coefficient information preservation

2. **Security**:
   - Randomness of mixing coefficients
   - Irreversibility without secret
   - Resistance to forgery

3. **Efficiency**:
   - 2x size reduction per layer
   - Logarithmic number of layers
   - Efficient FFT computations

## Application in STARK

1. **FRI Preparation**:
   - Mixing prepares data for FRI
   - Reduces proof size
   - Maintains verifiability

2. **Integration with DEEP**:
   - Compatibility with diagonal evaluation
   - Additional optimization
   - Overall efficiency improvement

In the next lesson, we'll examine how this mixed data is used in the commit phase of the FRI protocol.